# blazers - Extended Feature Selection
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* Feb 25, 2022

## Hypothesis
Two very important components of a machine learning model are feature selection and feature engineering. Our idea is that adding some more features to the StellarAlgo retention model could improve performance of the model. This notebook will test the standard StellarAlgo retention model features.

## Experiment
This section details our experiment including querying data, data transformations, feature selection and modelling.

In [1]:
import getpass
import pyodbc
import pandas as pd
import warnings
import matplotlib.pyplot as plt

from pycaret.classification import *

warnings.filterwarnings('ignore')

### Let's connect to MSSQL and run a stored proc to get our dataset:

In [2]:
# connect to SQL Server.
SERVER = '34.206.73.189'  
DATABASE = 'datascience' 
USERNAME = 'nrad' 
PASSWORD = getpass.getpass(prompt='Enter your password')
CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)

In [3]:
lkupclientid = 5 # blazers

cursor = CNXN.cursor()

storedProc = (
    f"""Exec [stlrTrailBlazers].[ds].[getRetentionScoringModelData] {lkupclientid}"""
)

df = pd.read_sql(storedProc, CNXN)

# apply some data transformations
df["year"] = pd.to_numeric(df["year"])

CNXN.commit()
cursor.close()

df.head()
#df.info()

,lkupClientId,dimCustomerMasterId,customerNumber,year,productGrouping,totalSpent,recentDate,attendancePercent,renewedBeforeDays,isBuyer,source_tenure,tenure,distToVenue,totalGames,recency,missed_games_1,missed_games_2,missed_games_over_2,click_link,fill_out_form,open_email,send_email,unsubscribe_email,openToSendRatio,clickToSendRatio,clickToOpenRatio,posting_records,resale_records,resale_atp,forward_records,cancel_records,email,inbound_email,inbound_phonecall,inperson_contact,internal_note,left_message,outbound_email,outbound_phonecall,phonecall,text,unknown,gender,childrenPresentInHH,maritalStatus,lengthOfResidenceInYrs,annualHHIncome,education,urbanicity,credits_after_refund,is_Lockdown,NumberofGamesPerSeason,CNTPostponedGames,isNextYear_Buyer
0,5,96,190309117,2017,Full Season,1672.00,2018-04-11,0.779070,233,TRUE,4015,423,None,38,0,9,3,0,48,0,264,700,0,0.377143,0.068571,0.181818,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,173,None,1
1,5,1335,1074005,2017,Full Season,1584.00,2018-04-11,0.813953,239,TRUE,5475,429,None,35,0,3,3,0,88,12,600,780,0,0.769231,0.112821,0.146667,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,,1,1,None,None,None,None,0.0,0,173,None,1
2,5,3366,203118752,2017,Mini Plan,646.00,2018-02-11,1.000000,52,TRUE,2190,153,None,4,0,0,0,0,48,0,60,100,0,0.600000,0.480000,0.800000,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,173,None,0
3,5,5444,201702156,2017,Quarter Season,1584.00,2018-04-01,0.979167,171,TRUE,2555,326,None,12,0,1,0,0,52,0,140,392,0,0.357143,0.132653,0.371429,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,173,None,0
4,5,5684,198195,2017,Full Season,11901.12,2018-04-11,0.994186,239,TRUE,2920,429,None,43,0,2,0,0,72,4,160,792,0,0.202020,0.090909,0.450000,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,173,None,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27495 entries, 0 to 27494
Data columns (total 54 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   lkupClientId            27495 non-null  int64  
 1   dimCustomerMasterId     27495 non-null  int64  
 2   customerNumber          27495 non-null  object 
 3   year                    27495 non-null  int64  
 4   productGrouping         27495 non-null  object 
 5   totalSpent              27495 non-null  float64
 6   recentDate              27495 non-null  object 
 7   attendancePercent       27495 non-null  float64
 8   renewedBeforeDays       27495 non-null  int64  
 9   isBuyer                 27495 non-null  object 
 10  source_tenure           27495 non-null  int64  
 11  tenure                  27495 non-null  int64  
 12  distToVenue             0 non-null      object 
 13  totalGames              27495 non-null  int64  
 14  recency                 27495 non-null

### Let's also drop the features that only have a single value, as they won't add much differentiation to our model:

In [5]:
for col in df.columns:
    if len(df[col].unique()) == 1:
        df.drop(col,inplace=True,axis=1)
        
df.shape

(27495, 28)

### We should also drop features that have a low correlation with the target label as they won't be useful for prediction, we'll only keep features that have a correlation above a set threshold:

In [6]:
cor = df.corr()

threshold = 0.05

#Correlation with output variable
cor_target = abs(cor["isNextYear_Buyer"])

#Selecting highly correlated features
relevant_features = cor_target[cor_target > threshold]

feats = []
for name, val in relevant_features.items():
    feats.append(name)

df_correlated = df[feats]

df_correlated.shape

df_correlated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27495 entries, 0 to 27494
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   year                    27495 non-null  int64  
 1   totalSpent              27495 non-null  float64
 2   renewedBeforeDays       27495 non-null  int64  
 3   source_tenure           27495 non-null  int64  
 4   tenure                  27495 non-null  int64  
 5   totalGames              27495 non-null  int64  
 6   missed_games_1          27495 non-null  int64  
 7   missed_games_2          27495 non-null  int64  
 8   missed_games_over_2     27495 non-null  int64  
 9   click_link              27495 non-null  int64  
 10  fill_out_form           27495 non-null  int64  
 11  open_email              27495 non-null  int64  
 12  send_email              27495 non-null  int64  
 13  openToSendRatio         27495 non-null  float64
 14  NumberofGamesPerSeason  27495 non-null

### Now that we have the right features we can look at the correlations between them, if features are highly correlated with each other it might negatively impact the model:

In [7]:
corr = df_correlated.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

,year,totalSpent,renewedBeforeDays,source_tenure,tenure,totalGames,missed_games_1,missed_games_2,missed_games_over_2,click_link,fill_out_form,open_email,send_email,openToSendRatio,NumberofGamesPerSeason,isNextYear_Buyer
year,1.00,0.03,0.16,-0.02,0.76,-0.16,0.02,0.12,0.16,-0.08,-0.11,-0.11,-0.17,-0.04,0.02,-0.06
totalSpent,0.03,1.00,0.14,0.17,0.08,0.24,0.13,0.07,0.08,-0.02,-0.04,0.00,0.03,-0.05,-0.03,0.13
renewedBeforeDays,0.16,0.14,1.00,0.42,0.52,0.62,0.47,0.29,0.23,0.14,0.04,0.31,0.41,0.10,-0.06,0.52
source_tenure,-0.02,0.17,0.42,1.00,0.25,0.41,0.26,0.13,0.08,0.09,-0.01,0.17,0.22,0.05,-0.02,0.34
tenure,0.76,0.08,0.52,0.25,1.00,0.13,0.20,0.21,0.21,0.03,-0.06,0.09,0.10,0.09,-0.00,0.21
totalGames,-0.16,0.24,0.62,0.41,0.13,1.00,0.57,0.23,0.06,0.22,0.12,0.41,0.50,0.10,-0.05,0.56
missed_games_1,0.02,0.13,0.47,0.26,0.20,0.57,1.00,0.35,0.17,0.09,0.05,0.21,0.32,0.00,-0.06,0.39
missed_games_2,0.12,0.07,0.29,0.13,0.21,0.23,0.35,1.00,0.34,-0.00,-0.01,0.06,0.14,-0.05,-0.08,0.23
missed_games_over_2,0.16,0.08,0.23,0.08,0.21,0.06,0.17,0.34,1.00,-0.05,-0.02,-0.00,0.05,-0.07,-0.15,0.17
click_link,-0.08,-0.02,0.14,0.09,0.03,0.22,0.09,-0.00,-0.05,1.00,0.36,0.56,0.46,0.38,0.15,0.15


### In order to compare two sets of features, we need to create some datasets for training and evalution:


In [8]:
# select % of the data for training
df_train = df_correlated.sample(frac=0.8, random_state=786).reset_index(drop=True)

# create the eval datasets for A and B
df_eval = df_correlated.drop(df_train.index).reset_index(drop=True)

# print out the number of records for training and eval
print('Data for Modeling: ' + str(df_train.shape))
print('Unseen Data For Predictions: ' + str(df_eval.shape), end="\n\n")

Data for Modeling: (21996, 16)
Unseen Data For Predictions: (5499, 16)



### Now we can model the data using a binary classification prediction for the isnextyear_buyer field to see how likely a customer is to re-purchase.

In [9]:
setup(
    data= df_train, 
    target="isNextYear_Buyer", 
    train_size = 0.80,
    data_split_shuffle=True,
    silent=True,
    numeric_features=[ 
        "totalSpent",
        "renewedBeforeDays",
        "tenure",
        "source_tenure",
        "totalGames",
        "missed_games_1",
        "missed_games_2",
        "missed_games_over_2",
        "click_link",
        "fill_out_form",
        "open_email",
        "send_email",
        "openToSendRatio",
        "NumberofGamesPerSeason"
    ]
)

,Description,Value
0,session_id,6315
1,Target,isNextYear_Buyer
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(21996, 16)"
5,Missing Values,False
6,Numeric Features,14
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


([],
 <MLUsecase.CLASSIFICATION: 1>,
 False,
 False,
 False,
 0        1
 1        1
 2        1
 3        1
 4        1
         ..
 21991    1
 21992    1
 21993    0
 21994    0
 21995    0
 Name: isNextYear_Buyer, Length: 21996, dtype: int64,
          totalSpent  renewedBeforeDays  source_tenure  tenure  totalGames  \
 16291    346.000000               35.0         2555.0   107.0         3.0   
 8038   12729.599609              224.0         5475.0  1128.0        20.0   
 11864   8132.799805              224.0        12410.0  1128.0        32.0   
 20289    638.000000               58.0         2920.0   833.0         2.0   
 12660  19176.000000              224.0         8395.0  1128.0        32.0   
 ...             ...                ...            ...     ...         ...   
 15611  12581.799805              270.0         2920.0  1889.0        22.0   
 5099    6023.899902              265.0         5110.0  1846.0        16.0   
 13905   2340.800049              220.0         365

In [10]:
model_matrix = compare_models(
    fold=10,
    include=["lr", "xgboost"]
)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.8374,0.9120,0.9091,0.8491,0.8781,0.6347,0.6381,0.9610
lr,Logistic Regression,0.8027,0.8659,0.8732,0.8296,0.8508,0.5601,0.5619,0.7590


In [11]:
best_model = create_model(model_matrix)
final_model = finalize_model(best_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8341,0.9140,0.9074,0.8462,0.8757,0.6271,0.6306
1,0.8392,0.9162,0.9109,0.8502,0.8795,0.6387,0.6422
2,0.8369,0.9088,0.9171,0.8435,0.8787,0.6312,0.6363
3,0.8364,0.9035,0.9127,0.8456,0.8779,0.6311,0.6353
4,0.8216,0.8989,0.8995,0.8361,0.8666,0.5984,0.6020
5,0.8426,0.9132,0.9109,0.8543,0.8817,0.6474,0.6504
6,0.8567,0.9268,0.9135,0.8705,0.8915,0.6811,0.6828
7,0.8562,0.9230,0.9206,0.8648,0.8918,0.6779,0.6808
8,0.8294,0.9145,0.9029,0.8434,0.8721,0.6170,0.6203
9,0.8204,0.9015,0.8959,0.8368,0.8653,0.5966,0.5998


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

## Observations
Here you can document some ideas on the results from above

## Conclusions
Here you can talk about next steps, did the experiment work? If yes, what to do next? If no, why?